<a href="https://colab.research.google.com/github/YMGYM/TSE_Learning/blob/master/Korean_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

이 파일은 딥러닝 네트워크로 한국어 언어 처리를 연습해보기 위한 파일입니다.

# import
필요한 파일, 라이브러리를 임포트합니다.
[AI HUB](https://aihub.or.kr/keti_data_board/language_intelligence) 데이터셋을 사용했습니다.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/content/drive/My Drive/Datasets/preprocessed_twitter_dialog_korean_dataset.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,31

In [3]:
dropped_data = data.drop(labels='Unnamed: 0', axis=1)

In [4]:
dropped_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403
0,기계식,키보드,쓰,어,보,고,싶,어요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,쌓,ㄴ,것,도,많,던데,쓰,어,보,세요,.,정말,좋,아요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,쌓,ㄴ,것,도,있,나요,?,예전,에,보,았,을,때,에는,다,10,만,원,이,넘,던데요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,이거,쓰,는데,예쁘,고,딸깍거리,어요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,우,와,진짜,예쁘,어요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13586,나,아도,고민,중,이,ㄴ데,가게,되,면,같이,가,아요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13587,좋,아요,.,그,때,는,돈,많이,벌,어,가야,이,겠,어요,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13588,좋,은,아파트,에,살,려,는,걸,무슨,욕망,의,노예,로,보,는,사람,들,이,종종,있,어요,.,그러,나,공공,시설,이,부족,하,ㄴ,한국,에서,아파트,는,너무나,살,기,좋,은,곳,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13589,개인적,으로,주차,공간,이,주택,이나,빌라,에,비하,면,많,다는,점,에서,도,아파트,를,희망,하,는,편,이,죠,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
filled_data = dropped_data.fillna(value="<unk>")

In [6]:
filled_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403
0,기계식,키보드,쓰,어,보,고,싶,어요,.,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,...,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>
1,쌓,ㄴ,것,도,많,던데,쓰,어,보,세요,.,정말,좋,아요,.,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,...,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>
2,쌓,ㄴ,것,도,있,나요,?,예전,에,보,았,을,때,에는,다,10,만,원,이,넘,던데요,.,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,...,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>
3,이거,쓰,는데,예쁘,고,딸깍거리,어요,.,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,...,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>
4,우,와,진짜,예쁘,어요,.,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,...,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>,<unk>


# Import Tokenizer

In [7]:
class Tokenizer:
  def __init__(self):
    self.word2idx = {}
    self.idx2word = {}

  def make_word_index(self, data):
    for vocab in data:
      if self.word2idx.get(vocab) == None: # get을 쓰지 않으면 Null 키 오류
        self.word2idx[vocab] = len(self.word2idx) + 1
        self.idx2word[len(self.idx2word) + 1] = vocab
  
  def get_data(self):
    return self.word2idx, self.idx2word

  def get_word(self, sent):
    result = np.array([],dtype="int16")
    for idx in sent:
      result = np.hstack([result, self.idx2word.get(idx)])
    return result

  def get_idx(self, sent):
    result = np.array([],dtype="int16")
    for word in sent:
      result = np.hstack([result, self.word2idx.get(word)])
    return result

In [8]:
tokenizer = Tokenizer()

In [9]:
# 데이터셋의 모든 줄을 반복해 데이터셋을 만듭니다.
for idx, row in filled_data.iterrows():
  tokenizer.make_word_index(row.dropna())

## Tokenizing Test

In [10]:
sent = filled_data.iloc[1]
result = tokenizer.get_idx(sent)
result


array([11, 12, 13, 14, 15, 16,  3,  4,  5, 17,  9, 18, 19, 20,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10

In [11]:
result2 = tokenizer.get_word(result)
result2

array(['쌓', 'ㄴ', '것', '도', '많', '던데', '쓰', '어', '보', '세요', '.', '정말', '좋',
       '아요', '.', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',
       '<unk>

# All data tokenize

In [12]:
temp = filled_data.to_numpy()
temp = temp[:,:50]
shorten_data = pd.DataFrame(temp)

In [13]:
tokenized = pd.DataFrame()
for idx, row in shorten_data.iterrows():
  tokenized = tokenized.append(pd.DataFrame(tokenizer.get_idx(row).reshape(1,-1)), ignore_index=True)

In [14]:
tokenized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1,2,3,4,5,6,7,8,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
1,11,12,13,14,15,16,3,4,5,17,9,18,19,20,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
2,11,12,13,14,21,22,23,24,25,5,26,27,28,29,30,31,32,33,34,35,36,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
3,37,3,38,39,6,40,8,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
4,41,42,43,39,8,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13586,101,99,292,146,34,373,397,114,46,227,70,20,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
13587,19,20,9,240,28,72,464,358,1160,4,462,34,94,8,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
13588,19,130,1715,25,251,1175,72,441,456,6278,52,4597,256,5,72,88,122,34,3043,21,8,9,183,101,10031,3070,34,506,96,12,1262,225,1715,72,910,251,71,19,130,543,34,89,9,9951,119,1716,119,10032,70,608
13589,233,172,10035,2444,34,2328,401,8046,25,2458,46,15,1237,824,225,14,1715,76,3143,96,72,213,34,355,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10


In [15]:
np_tokenized = tokenized.to_numpy()

In [16]:
np_tokenized.shape

(13591, 50)

# Reverse Padding

In [17]:
total = list()
for row in np_tokenized[:500]:
  for col in range(0, len(row)):
    if row[col] == tokenizer.word2idx['<unk>']:
      continue
    else:
      total.append(row[:col+1].tolist())

In [18]:
padded_total = K.preprocessing.sequence.pad_sequences(total, maxlen=50)

In [19]:
padded_total.shape

(8920, 50)

# make data y

In [20]:
xs, ys = padded_total[:, :-1], padded_total[:, -1:]

In [21]:
xs.shape

(8920, 49)

In [22]:
ys.shape

(8920, 1)

# Make Data One-Hot

In [23]:
ys.shape

(8920, 1)

In [24]:
ys = K.utils.to_categorical(ys)

In [25]:
ys.shape

(8920, 1440)

# Make Sequential Model

In [26]:
model = K.Sequential()
model.add(K.layers.Embedding(ys.shape[1], 256))
model.add(K.layers.LSTM(256, return_sequences=False))
model.add(K.layers.Dense(ys.shape[1], activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         368640    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 1440)              370080    
Total params: 1,264,032
Trainable params: 1,264,032
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(optimizer = "adam", loss="categorical_crossentropy")

In [43]:
model.fit(x=xs, y=ys, batch_size = 128, epochs=30)

Epoch 1/30
70/70 [==============================] - 37s 528ms/step - loss: 1.6577
Epoch 2/30
70/70 [==============================] - 37s 526ms/step - loss: 1.5984
Epoch 3/30
70/70 [==============================] - 37s 524ms/step - loss: 1.5421
Epoch 4/30
70/70 [==============================] - 37s 527ms/step - loss: 1.4911
Epoch 5/30
70/70 [==============================] - 37s 528ms/step - loss: 1.4411
Epoch 6/30
70/70 [==============================] - 37s 522ms/step - loss: 1.3933
Epoch 7/30
70/70 [==============================] - 37s 523ms/step - loss: 1.3454
Epoch 8/30
70/70 [==============================] - 37s 524ms/step - loss: 1.3017
Epoch 9/30
70/70 [==============================] - 38s 542ms/step - loss: 1.2597
Epoch 10/30
70/70 [==============================] - 37s 524ms/step - loss: 1.2187
Epoch 11/30
70/70 [==============================] - 37s 525ms/step - loss: 1.1796
Epoch 12/30
70/70 [==============================] - 37s 529ms/step - loss: 1.1438
Epoch 13/30
7

In [36]:
data = model.predict(x = xs[5:6]).argmax()

In [104]:
def project_sentence(start_word, len=49):
  sentence = np.zeros(shape=(1, 49 ))
  sentence = np.append(sentence, start_word)
  for i in range(0, len-1):
    predict = model.predict(sentence.reshape(1,-1))
    data = predict.argmax()
    sentence = sentence[1:]
    sentence = np.append(sentence, data)

  predict = model.predict(sentence.reshape(1,-1))
  sentence = np.append(sentence, predict.argmax())
  return sentence

In [105]:
data = project_sentence(15)
data

array([   0.,   15.,  130.,   86.,   34.,   89.,    9.,   57.,  756.,
       1286.,  416., 1290.,  697.,  184.,   63.,  421.,   21.,   27.,
         13.,  129.,   20.,    9., 1153., 1154.,   82.,  220., 1155.,
        355.,   23.,   51.,   52.,  962.,  963.,  949.,  344.,  218.,
          8.,    9.,   59.,  654.,  201.,  113.,   96.,    6.,  446.,
        130.,  714.,  637.,   72.,   13.,   34.])

In [108]:
sent = ""
for i in data:
  if i != 0:
    sent += (tokenizer.idx2word[i] + " ")
    # print(tokenizer.idx2word[i])

print(sent)

많 은 거 이 에요 . 그리하 여도 구마 님 충분히 잘하 시 ㄹ 수 있 을 것 같 아요 . 실력 녹슬 었 으면 어떡하 죠 ? 저 의 친 구도 알약 못 먹 어요 . 근데 새벽 세 게 하 고 노래방 은 자주 마시 는 것 이 


In [97]:
model.predict(xs[6:7]).argmax()

7

In [99]:
tokenizer.idx2word[7]

'싶'

In [ ]:
for 